In [36]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrowfs_adlgen2 as pa_adl

from dotenv import load_dotenv
from pathlib import Path

from azure.identity import DefaultAzureCredential, ClientSecretCredential
from azure.storage.filedatalake import DataLakeServiceClient
from azure.keyvault.secrets import SecretClient
from azure.core.exceptions import ResourceNotFoundError

# local

In [3]:
# DATA_DIR = "../include/data"

In [4]:
# # local
# BRONZE_FOLDER_NAME = "bronze"
# BRONZE_DATA_DIR = os.path.join("{DATA_DIR}", "{FOLDER_NAME}").replace("\\", "/")
# BRONZE_DATA_DIR

In [5]:
# # local
# file_infos = [
#     os.path.join(BRONZE_DATA_DIR.format(DATA_DIR=DATA_DIR, FOLDER_NAME=BRONZE_FOLDER_NAME), file_info).replace("\\", "/") 
#     for file_info in 
#     os.listdir(BRONZE_DATA_DIR.format(DATA_DIR=DATA_DIR, FOLDER_NAME=BRONZE_FOLDER_NAME))
# ]
# file_infos

In [6]:
# label_paths = [
#     os.path.join(file_info, "etc", "README").replace("\\", "/") 
#     for file_info in file_infos 
#     if os.path.exists(os.path.join(file_info, "etc", "README").replace("\\", "/"))
# ]
# label_paths

# cloud (which will includde connecting to azure data lake storage via duck db)

In [7]:
# Retrieve credentials from environment variables
# this is strictly used only in development
# load env variables
env_dir = Path('../').resolve()
load_dotenv(os.path.join(env_dir, '.env'))

True

In [8]:
storage_account_name = os.environ.get("STORAGE_ACCOUNT_NAME")
credential = os.environ.get("STORAGE_ACCOUNT_KEY")
conn_str = os.environ.get("STORAGE_ACCOUNT_CONN_STR")

In [9]:
# cloud
URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net"
BRONZE_FOLDER_NAME = f"{storage_account_name}-bronze"
BRONZE_DATA_DIR = URL.format(
    FOLDER_NAME=BRONZE_FOLDER_NAME
)
BRONZE_DATA_DIR

'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net'

In [10]:
# cloud
# create client with generated sas token
datalake_service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net", 
    credential=credential
)

# retrieves file system client/container client 
# to retrieve datalake client
bronze_container_client = datalake_service_client.get_file_system_client(f"{storage_account_name}-bronze")

# we only get the directories in the first level of 
# the container, if it has a "/" then it means it is not
# an immediate folder in the container. This only really
# gets the subject folders 
subject_folders = [path.name for path in bronze_container_client.get_paths() if not "/" in path.name]

In [11]:
file_infos = [os.path.join(BRONZE_DATA_DIR, subject_folder).replace("\\", "/") for subject_folder in subject_folders]
file_infos

['abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1028-20100710-hne',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-ajg',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-tkg',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1snoke-20120412-hge',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/23yipikaye-20100807-ujm',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Aaron-20080318-kdl',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-bft',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-cqj',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-fcp',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-hns',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-nky',
 'abfss://sgppipeline

In [12]:
# need some way to ensure the path also exists
label_paths = [
    os.path.join(file_info, "etc", "README").replace("\\", "/") 
    for i, file_info in enumerate(file_infos) 
    if bronze_container_client
    .get_file_client(file_path=f"{subject_folders[i]}/etc/README")
    .exists()
]
label_paths

['abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1028-20100710-hne/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-ajg/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-tkg/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1snoke-20120412-hge/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/23yipikaye-20100807-ujm/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Aaron-20080318-kdl/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-bft/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-cqj/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-fcp/etc/README',
 'abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/Anniepoo-20140308-hns/etc/README',
 

In [13]:
conn = duckdb.connect()

In [14]:
# for cloud only
# installing dependencies and creating secrets object
conn.sql(f"""INSTALL azure""")
conn.sql(f"""LOAD azure""")
conn.sql(f"""
    CREATE OR REPLACE SECRET az_sgp (
        TYPE azure,
        CONNECTION_STRING '{conn_str}'
    );
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [15]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE split_raw_labels AS (
        WITH raw_labels AS (
            SELECT 
                filename, 
                content 
            FROM read_text({label_paths})
        )

        SELECT 
            filename, 
            REGEXP_SPLIT_TO_TABLE(content, '\n\n*') AS value 
        FROM raw_labels
    )
""")

In [16]:
conn.sql("""
    SELECT *  FROM split_raw_labels 
""")

┌───────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────┐
│                                           filename                                            │                              value                               │
│                                            varchar                                            │                             varchar                              │
├───────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1028-20100710-hne/etc/README  │ User Name:1028                                                   │
│ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1028-20100710-hne/etc/README  │ Speaker Characteristics:                                         │
│ abfss://

# clean value column containing the label

In [17]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE subjects_labels AS (
        -- remove the rows with no gender meta data of the subject
        WITH subjects_labels_1 AS (
            SELECT * 
            FROM split_raw_labels
            WHERE 'gender' IN LCASE(value)
        ),

        subjects_labels_2 AS (
            SELECT
                -- remove punctuations in the meta data of the subject
                -- containing its gender
                REGEXP_REPLACE(LCASE(value), '[:;\[\]\t\n\s]+', '') AS value,
                filename
            FROM subjects_labels_1
        ),

        subjects_labels_3 AS (
            SELECT
                -- remove 'gender' in meta data of the subject containing
                -- its gender
                REGEXP_REPLACE(value, 'gender', '') AS value,
                filename
            FROM subjects_labels_2
        ),
            
        subjects_labels_4 AS (
            SELECT
                CASE
                    WHEN STARTS_WITH(value, 'ma') OR STARTS_WITH(value, 'mä') THEN 'male'
                    WHEN STARTS_WITH(value, 'fem') OR STARTS_WITH(value, 'wei') THEN 'female'
            
                    -- whenever unknown gender is encountered just generate random
                    -- gender of either male or female with prob of 50%
                    ELSE 
                        (CASE 
                            WHEN RANDOM() < 0.5 THEN 'male'
                            ELSE 'female'
                        END)
                END AS value,
                filename
            FROM subjects_labels_3
        )
            
        SELECT * FROM subjects_labels_4
    )
""")

In [18]:
conn.sql("""
    SELECT * FROM subjects_labels
""")

┌─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  value  │                                              filename                                              │
│ varchar │                                              varchar                                               │
├─────────┼────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ male    │ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1028-20100710-hne/etc/README       │
│ female  │ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-ajg/etc/README     │
│ female  │ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1337ad-20170321-tkg/etc/README     │
│ male    │ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/1snoke-20120412-hge/etc/README     │
│ male    │ abfss://sgppipelinesa-bronze@sgppipelinesa.dfs.core.windows.net/23yipikaye-20100807-

# clean filename column and turn into subjectid column

In [19]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE subjects_labels AS (
        SELECT
            -- splits the file path on the '/' character which
            -- results in a list that we can use to extract the
            -- third to the last value in this list
            LIST_EXTRACT(STRING_SPLIT(filename, '/'), -3) AS subjectId,
            value
        FROM subjects_labels
    )
""")

In [20]:
conn.sql("""
    SELECT * FROM subjects_labels
""")

┌─────────────────────────┬─────────┐
│        subjectId        │  value  │
│         varchar         │ varchar │
├─────────────────────────┼─────────┤
│ 1028-20100710-hne       │ male    │
│ 1337ad-20170321-ajg     │ female  │
│ 1337ad-20170321-tkg     │ female  │
│ 1snoke-20120412-hge     │ male    │
│ 23yipikaye-20100807-ujm │ male    │
│ Aaron-20080318-kdl      │ male    │
│ Anniepoo-20140308-bft   │ female  │
│ Anniepoo-20140308-cqj   │ female  │
│ Anniepoo-20140308-fcp   │ female  │
│ Anniepoo-20140308-hns   │ female  │
│ Anniepoo-20140308-nky   │ female  │
│ Coren-20141121-pxp      │ male    │
├─────────────────────────┴─────────┤
│ 12 rows                 2 columns │
└───────────────────────────────────┘

# once the `value`'s and `subjectId`'s is cleaned we can now split the dataset uniformly. We need to split first on the male and female subjects so that we can evenly split these male and female sets into train, val, and test sets, and then later rejoin the train, val, and test sets of the male and female sets  

In [21]:
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE male_subjects_labels AS (
        SELECT 
            *, 
            ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id 
        FROM subjects_labels
        WHERE value = 'male'
    );
    
    CREATE OR REPLACE TEMPORARY TABLE female_subjects_labels AS (
        SELECT 
            *, 
            ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id 
        FROM subjects_labels
        WHERE value = 'female'
    );    
""")

# create a 70, 15, 15 ratio for the training, validation, and testing sets

In [22]:
# creates 70, 15, 15 ratio for male labels
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE male_subjects_labels_split AS (
        -- split the male subjects labels into train and test 
        WITH male_subjects_labels_train__ AS (
            SELECT 
                *,
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.7 FROM male_subjects_labels) THEN 'train'
                    ELSE 'test'
                END AS split
            FROM male_subjects_labels
        ),
        
        -- recalculate the test splits split row id so it 
        -- starts from 1 to n again 
        temp AS (
            SELECT 
                * EXCLUDE(split_row_id), 
                ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id
            FROM male_subjects_labels_train__
            WHERE split = 'test'
        ),
        
        -- split again the previous test set into now the real
        -- validation and testing sets
        male_subjects_labels_val_test AS (
            SELECT
                * EXCLUDE(split),
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.5 FROM temp) THEN 'val'
                    ELSE 'test'
                END AS split
            FROM temp
        )
        
        -- unionize the male subjects labels with validation
        -- and testing splits and the previous male subjects
        -- labels with the train splits 
        SELECT * FROM male_subjects_labels_val_test
        UNION BY NAME
        SELECT * FROM male_subjects_labels_train__
        WHERE split = 'train'
    )
""")

In [23]:
conn.sql("""
    SELECT * FROM male_subjects_labels_split
""")

┌─────────────────────────┬─────────┬──────────────┬─────────┐
│        subjectId        │  value  │ split_row_id │  split  │
│         varchar         │ varchar │    int64     │ varchar │
├─────────────────────────┼─────────┼──────────────┼─────────┤
│ 23yipikaye-20100807-ujm │ male    │            3 │ train   │
│ 1snoke-20120412-hge     │ male    │            2 │ train   │
│ Coren-20141121-pxp      │ male    │            2 │ test    │
│ 1028-20100710-hne       │ male    │            1 │ train   │
│ Aaron-20080318-kdl      │ male    │            1 │ val     │
└─────────────────────────┴─────────┴──────────────┴─────────┘

In [24]:
# creates 70, 15, 15 ratio for female labels
conn.sql("""
    CREATE OR REPLACE TEMPORARY TABLE female_subjects_labels_split AS (
        -- split the female subjects labels into train and test 
        WITH female_subjects_labels_train__ AS (
            SELECT 
                *,
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.7 FROM female_subjects_labels) THEN 'train'
                    ELSE 'test'
                END AS split
            FROM female_subjects_labels
        ),
        
        -- recalculate the test splits split row id so it 
        -- starts from 1 to n again 
        temp AS (
            SELECT 
                * EXCLUDE(split_row_id), 
                ROW_NUMBER() OVER(ORDER BY subjectId) AS split_row_id
            FROM female_subjects_labels_train__
            WHERE split = 'test'
        ),
        
        -- split again the previous test set into now the real
        -- validation and testing sets
        female_subjects_labels_val_test AS (
            SELECT
                * EXCLUDE(split),
                CASE
                    WHEN split_row_id <= (SELECT COUNT(*) * 0.5 FROM temp) THEN 'val'
                    ELSE 'test'
                END AS split
            FROM temp
        )
        
        -- unionize the female subjects labels with validation
        -- and testing splits and the previous female subjects
        -- labels with the train splits 
        SELECT * FROM female_subjects_labels_val_test
        UNION BY NAME
        SELECT * FROM female_subjects_labels_train__
        WHERE split = 'train'
    )
""")

In [25]:
conn.sql("""
    SELECT * FROM female_subjects_labels_split
""")

┌───────────────────────┬─────────┬──────────────┬─────────┐
│       subjectId       │  value  │ split_row_id │  split  │
│        varchar        │ varchar │    int64     │ varchar │
├───────────────────────┼─────────┼──────────────┼─────────┤
│ 1337ad-20170321-tkg   │ female  │            2 │ train   │
│ Anniepoo-20140308-bft │ female  │            3 │ train   │
│ Anniepoo-20140308-fcp │ female  │            1 │ val     │
│ Anniepoo-20140308-nky │ female  │            3 │ test    │
│ Anniepoo-20140308-hns │ female  │            2 │ test    │
│ 1337ad-20170321-ajg   │ female  │            1 │ train   │
│ Anniepoo-20140308-cqj │ female  │            4 │ train   │
└───────────────────────┴─────────┴──────────────┴─────────┘

# rejoin/unionize the male and female subjects labels based on their split as well as recreate new `rowId`'s for each split and create a partition column for each split

In [26]:
n_partitions = 10

In [28]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE train_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'train'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'train'
        ),
         
        -- create new rowIds for the split
        train_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM train_labels
    );
""")

In [29]:
train_labels_table = conn.sql("""
    SELECT * FROM train_labels
""").to_arrow_table()
train_labels_table

pyarrow.Table
subjectId: string
value: string
split: string
rowId: int64
partition: int64
----
subjectId: [["1028-20100710-hne","1337ad-20170321-ajg","1337ad-20170321-tkg","1snoke-20120412-hge","23yipikaye-20100807-ujm","Anniepoo-20140308-bft","Anniepoo-20140308-cqj"]]
value: [["male","female","female","male","male","female","female"]]
split: [["train","train","train","train","train","train","train"]]
rowId: [[0,1,2,3,4,5,6]]
partition: [[0,1,2,3,4,5,6]]

In [30]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE val_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'val'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'val'
        ),
         
        -- create new rowIds for the split
        val_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM val_labels
    );
""")

In [32]:
val_labels_table = conn.sql("""         
    SELECT * FROM val_labels
""").to_arrow_table()
val_labels_table

pyarrow.Table
subjectId: string
value: string
split: string
rowId: int64
partition: int64
----
subjectId: [["Aaron-20080318-kdl","Anniepoo-20140308-fcp"]]
value: [["male","female"]]
split: [["val","val"]]
rowId: [[0,1]]
partition: [[0,1]]

In [33]:
conn.sql(f"""
    CREATE OR REPLACE TEMPORARY TABLE test_labels AS (
        WITH subjects_labels_split AS (
            SELECT subjectId, value, split
            FROM female_subjects_labels_split
            WHERE split = 'test'
            
            UNION BY NAME
            
            SELECT subjectId, value, split
            FROM male_subjects_labels_split
            WHERE split = 'test'
        ),
         
        -- create new rowIds for the split
        test_labels AS (
            SELECT
                *,
                ROW_NUMBER() OVER(ORDER BY subjectId) - 1 AS rowId
            FROM subjects_labels_split
        )
         
        -- creawte partition column
        SELECT
            *,
            rowId % {n_partitions} AS partition
        FROM test_labels
    );
""")

In [34]:
test_labels_table = conn.sql("""
    SELECT * FROM test_labels
""").to_arrow_table()
test_labels_table

pyarrow.Table
subjectId: string
value: string
split: string
rowId: int64
partition: int64
----
subjectId: [["Anniepoo-20140308-hns","Anniepoo-20140308-nky","Coren-20141121-pxp"]]
value: [["female","female","male"]]
split: [["test","test","test"]]
rowId: [[0,1,2]]
partition: [[0,1,2]]

# save the data to azure data lake or locally

In [ ]:
# # local
# SILVER_FOLDER_NAME = "silver"
# SUB_FOLDER_NAME = "stage-01"
# SILVER_DATA_DIR = os.path.join("{DATA_DIR}", "{FOLDER_NAME}", "{SUB_FOLDER_NAME}").replace("\\", "/")
# SILVER_DATA_DIR

In [ ]:
# SAVE_DIR = SILVER_DATA_DIR.format(
#     DATA_DIR=DATA_DIR,
#     FOLDER_NAME=SILVER_FOLDER_NAME,
#     SUB_FOLDER_NAME=SUB_FOLDER_NAME
# )
# os.makedirs(SAVE_DIR, exist_ok=True)
# SAVE_DIR

In [42]:
# cloud
# URL = "abfss://{FOLDER_NAME}@sgppipelinesa.dfs.core.windows.net"
URL = "{FOLDER_NAME}"
SILVER_FOLDER_NAME = "sgppipelinesa-silver"
SUB_FOLDER_NAME = "stage-01"
SILVER_DATA_DIR = os.path.join(URL, "{SUB_FOLDER_NAME}").replace("\\", "/")
SILVER_DATA_DIR

'{FOLDER_NAME}/{SUB_FOLDER_NAME}'

In [43]:
SAVE_DIR = SILVER_DATA_DIR.format(
    FOLDER_NAME=SILVER_FOLDER_NAME,
    SUB_FOLDER_NAME=SUB_FOLDER_NAME
)
SAVE_DIR

'sgppipelinesa-silver/stage-01'

# Unfortunately writing to azure using duckdb is not possible yet so workaround is to use pyarrow table

In [ ]:
# conn.sql(f"""
#     COPY (SELECT * FROM train_labels) TO '{SAVE_DIR}/train_labels.parquet' (FORMAT parquet, PARTITION_BY partition, OVERWRITE)
# """)

NotImplementedException: Not implemented Error: AzureBlobStorageFileSystem: DirectoryExists is not implemented!

In [45]:
handler = pa_adl.AccountHandler.from_account_name(storage_account_name, credential=credential)
fs = pa.fs.PyFileSystem(handler)

In [47]:
pq.write_table(train_labels_table, f"{SAVE_DIR}/train_labels.parquet", filesystem=fs)
pq.write_table(val_labels_table, f"{SAVE_DIR}/val_labels.parquet", filesystem=fs)
pq.write_table(test_labels_table, f"{SAVE_DIR}/test_labels.parquet", filesystem=fs)